In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from pycaret.regression import setup, compare_models
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import numpy as np
import pandas as pd
import multiprocessing

In [2]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge, OrthogonalMatchingPursuit, LinearRegression, Lasso, Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Read train and test sets

In [3]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

# Store both dfs lengths

In [4]:
m_train = df_train.shape[0]
m_test = df_test.shape[0]
m_train, m_test

(1460, 1459)

# Concat the dfs

In [5]:
df = pd.concat([df_train, df_test])
assert df.shape[0] == m_train + m_test

In [6]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


# Save target

In [7]:
target_col, target = "SalePrice", df["SalePrice"]

#  Drop Id and target columns

In [8]:
cols_to_drop = ["Id", target_col]
df.drop(cols_to_drop, axis=1, inplace=True)

# View and fix columns data-types

In [9]:
df.select_dtypes(object).columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [10]:
df.select_dtypes(np.number).columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')

In [11]:
num_to_obj_cols = ['MSSubClass', 'MoSold']
df[num_to_obj_cols] = df[num_to_obj_cols].astype(object)

In [12]:
cols_cat = df.select_dtypes(object).columns.to_list()
cols_num = df.select_dtypes(np.number).columns.to_list()

# Impute categorical columns

In [13]:
cols_cat_na = df[cols_cat].isnull().sum()[df[cols_cat].isnull().sum() > 0]
cols_cat_na

MSZoning           4
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinType2      80
Electrical         1
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageFinish     159
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
dtype: int64

In [14]:
mode_filled_cols = ["MSZoning", "Utilities", "Exterior1st", "Exterior2nd", "MasVnrType", "Electrical", "KitchenQual", "Functional", "SaleType"]
for col in mode_filled_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

none_filled_cols = ["Alley", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "FireplaceQu", "GarageType", "GarageFinish", "GarageQual", "GarageCond", "PoolQC", "Fence", "MiscFeature"]
for col in none_filled_cols:
    df[col].fillna("None", inplace=True)
    
df[cols_cat].isnull().sum().sum()

0

# Change ordinal columns to numeric, and encode accordingly

In [15]:
df.select_dtypes(object).columns

Index(['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC',
       'Fence', 'MiscFeature', 'MoSold', 'SaleType', 'SaleCondition'],
      dtype='object')

In [16]:
oe = OrdinalEncoder(categories=[['Reg', 'IR1', 'IR2', 'IR3']])
df.loc[:, "LotShape"] = oe.fit_transform(df[["LotShape"]])
df["LotShape_sq"] = df["LotShape"] ** 2

In [17]:
oe = OrdinalEncoder(categories=[['Gtl', 'Mod', 'Sev']])
df.loc[:, "LandSlope"] = oe.fit_transform(df[["LandSlope"]])
df["LandSlope_sq"] = df["LandSlope"] ** 2

In [18]:
qual_oe = OrdinalEncoder(categories=[['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex']])
for col in ["ExterQual", "ExterCond", 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']:
    df.loc[:, col] = qual_oe.fit_transform(df[[col]])
    df[col + "sq"] = df[col] ** 2

In [19]:
oe = OrdinalEncoder(categories=[['None', 'No', 'Mn', 'Av', 'Gd']])
df.loc[:, "BsmtExposure"] = oe.fit_transform(df[["BsmtExposure"]])
df["BsmtExposure_sq"] = df["BsmtExposure"] ** 2

In [20]:
oe = OrdinalEncoder(categories=[['None', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']])
for col in ["BsmtFinType1", "BsmtFinType2"]:
    df.loc[:, col] = oe.fit_transform(df[[col]])
    df[col + "sq"] = df[col] ** 2

In [21]:
oe = OrdinalEncoder(categories=[['None', 'Unf', 'RFn', 'Fin']])
df.loc[:, "GarageFinish"] = oe.fit_transform(df[["GarageFinish"]])
df["GarageFinishsq"] = df["GarageFinish"] ** 2

In [22]:
oe = OrdinalEncoder(categories=[['N', 'P', 'Y']])
df.loc[:, "PavedDrive"] = oe.fit_transform(df[["PavedDrive"]])
df["PavedDrivesq"] = df["PavedDrive"] ** 2

In [23]:
oe = OrdinalEncoder(categories=[['None', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']])
df.loc[:, "Fence"] = oe.fit_transform(df[["Fence"]])
df["Fencesq"] = df["Fence"] ** 2

In [24]:
cols_cat = df.select_dtypes(object).columns.to_list()
cols_num = df.select_dtypes(np.number).columns.to_list()

# Impute numerical columns

In [25]:
cols_num_na = df[cols_num].isnull().sum()[df[cols_num].isnull().sum() > 0]
cols_num_na

LotFrontage     486
MasVnrArea       23
BsmtFinSF1        1
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
GarageYrBlt     159
GarageCars        1
GarageArea        1
dtype: int64

In [26]:
zero_filled_cols = ["BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "BsmtFullBath", "BsmtHalfBath", "GarageCars", "GarageArea"]
for col in zero_filled_cols:
    df[col].fillna(0, inplace=True)

In [27]:
cols_num_na = df[cols_num].isnull().sum()[df[cols_num].isnull().sum() > 0]
cols_num_na

LotFrontage    486
MasVnrArea      23
GarageYrBlt    159
dtype: int64

In [28]:
def catboost_imputer(df, cols_num_na):
    
    # save columns to impute and drop them from df
    cols_to_impute = df[cols_num_na]
    df.drop(cols_num_na, axis=1, inplace=True)
    
    for col in cols_num_na:
        # define X and y
        X, y = df.copy(), cols_to_impute[col]

        # get train and test sets
        train_indexes, test_indexes = ~y.isnull(), y.isnull()
        X_train, X_test = X.loc[train_indexes, :], X.loc[test_indexes, :]
        y_train, y_test = y.loc[train_indexes], y.loc[test_indexes]
        
        model = CatBoostRegressor(max_depth=8, random_seed=10,
                                  subsample=0.65, n_estimators=1000,
                                  cat_features=cols_cat)
        model.fit(X_train, y_train)
        cols_to_impute.loc[test_indexes, col] = model.predict(X_test)
        df[col] = cols_to_impute[col]
    return df

In [29]:
df = catboost_imputer(df, cols_num_na.index)
df.isnull().sum().sum()

Learning rate set to 0.047118
0:	learn: 22.8559408	total: 87.2ms	remaining: 1m 27s
1:	learn: 22.3971548	total: 103ms	remaining: 51.2s
2:	learn: 21.9638828	total: 119ms	remaining: 39.4s
3:	learn: 21.5490582	total: 130ms	remaining: 32.4s
4:	learn: 21.0911381	total: 145ms	remaining: 28.8s
5:	learn: 20.7153817	total: 159ms	remaining: 26.4s
6:	learn: 20.2903763	total: 171ms	remaining: 24.2s
7:	learn: 19.9186439	total: 188ms	remaining: 23.4s
8:	learn: 19.5674084	total: 200ms	remaining: 22s
9:	learn: 19.2232179	total: 219ms	remaining: 21.7s
10:	learn: 18.9409731	total: 231ms	remaining: 20.7s
11:	learn: 18.6465703	total: 248ms	remaining: 20.4s
12:	learn: 18.4092670	total: 253ms	remaining: 19.2s
13:	learn: 18.1629147	total: 267ms	remaining: 18.8s
14:	learn: 17.8953425	total: 278ms	remaining: 18.3s
15:	learn: 17.6851373	total: 294ms	remaining: 18.1s
16:	learn: 17.4374575	total: 306ms	remaining: 17.7s
17:	learn: 17.2409281	total: 323ms	remaining: 17.6s
18:	learn: 17.0445667	total: 334ms	remaining

171:	learn: 10.5114655	total: 2.52s	remaining: 12.1s
172:	learn: 10.5069250	total: 2.54s	remaining: 12.1s
173:	learn: 10.4759288	total: 2.55s	remaining: 12.1s
174:	learn: 10.4623418	total: 2.56s	remaining: 12.1s
175:	learn: 10.4382841	total: 2.58s	remaining: 12.1s
176:	learn: 10.4367621	total: 2.59s	remaining: 12s
177:	learn: 10.4247440	total: 2.6s	remaining: 12s
178:	learn: 10.4162647	total: 2.62s	remaining: 12s
179:	learn: 10.4142624	total: 2.63s	remaining: 12s
180:	learn: 10.4105673	total: 2.65s	remaining: 12s
181:	learn: 10.3823237	total: 2.66s	remaining: 12s
182:	learn: 10.3781711	total: 2.68s	remaining: 12s
183:	learn: 10.3726403	total: 2.69s	remaining: 11.9s
184:	learn: 10.3555879	total: 2.71s	remaining: 11.9s
185:	learn: 10.3458521	total: 2.72s	remaining: 11.9s
186:	learn: 10.3418985	total: 2.74s	remaining: 11.9s
187:	learn: 10.3214380	total: 2.75s	remaining: 11.9s
188:	learn: 10.2907587	total: 2.77s	remaining: 11.9s
189:	learn: 10.2881401	total: 2.78s	remaining: 11.8s
190:	lea

329:	learn: 8.6547620	total: 5.04s	remaining: 10.2s
330:	learn: 8.6481842	total: 5.05s	remaining: 10.2s
331:	learn: 8.6388078	total: 5.07s	remaining: 10.2s
332:	learn: 8.6275171	total: 5.08s	remaining: 10.2s
333:	learn: 8.6177723	total: 5.1s	remaining: 10.2s
334:	learn: 8.6078695	total: 5.11s	remaining: 10.2s
335:	learn: 8.5939151	total: 5.13s	remaining: 10.1s
336:	learn: 8.5804470	total: 5.14s	remaining: 10.1s
337:	learn: 8.5795076	total: 5.16s	remaining: 10.1s
338:	learn: 8.5720860	total: 5.17s	remaining: 10.1s
339:	learn: 8.5570511	total: 5.19s	remaining: 10.1s
340:	learn: 8.5551819	total: 5.2s	remaining: 10.1s
341:	learn: 8.5447953	total: 5.22s	remaining: 10s
342:	learn: 8.5443489	total: 5.23s	remaining: 10s
343:	learn: 8.5432941	total: 5.25s	remaining: 10s
344:	learn: 8.5397365	total: 5.27s	remaining: 10s
345:	learn: 8.5391325	total: 5.29s	remaining: 9.99s
346:	learn: 8.5305335	total: 5.3s	remaining: 9.97s
347:	learn: 8.5299749	total: 5.32s	remaining: 9.97s
348:	learn: 8.5077548	t

496:	learn: 7.4213741	total: 7.79s	remaining: 7.89s
497:	learn: 7.4115983	total: 7.81s	remaining: 7.87s
498:	learn: 7.4109934	total: 7.83s	remaining: 7.86s
499:	learn: 7.3989521	total: 7.84s	remaining: 7.84s
500:	learn: 7.3979975	total: 7.86s	remaining: 7.83s
501:	learn: 7.3894945	total: 7.87s	remaining: 7.81s
502:	learn: 7.3830584	total: 7.89s	remaining: 7.8s
503:	learn: 7.3796488	total: 7.91s	remaining: 7.78s
504:	learn: 7.3788617	total: 7.92s	remaining: 7.77s
505:	learn: 7.3688350	total: 7.94s	remaining: 7.75s
506:	learn: 7.3677290	total: 7.96s	remaining: 7.74s
507:	learn: 7.3664569	total: 7.97s	remaining: 7.72s
508:	learn: 7.3653182	total: 8s	remaining: 7.71s
509:	learn: 7.3631805	total: 8.01s	remaining: 7.7s
510:	learn: 7.3563875	total: 8.04s	remaining: 7.69s
511:	learn: 7.3443853	total: 8.05s	remaining: 7.67s
512:	learn: 7.3438850	total: 8.07s	remaining: 7.66s
513:	learn: 7.3393110	total: 8.09s	remaining: 7.64s
514:	learn: 7.3245976	total: 8.1s	remaining: 7.63s
515:	learn: 7.3176

661:	learn: 6.3938606	total: 10.5s	remaining: 5.36s
662:	learn: 6.3853721	total: 10.5s	remaining: 5.35s
663:	learn: 6.3829856	total: 10.5s	remaining: 5.33s
664:	learn: 6.3768269	total: 10.6s	remaining: 5.32s
665:	learn: 6.3747678	total: 10.6s	remaining: 5.3s
666:	learn: 6.3744637	total: 10.6s	remaining: 5.29s
667:	learn: 6.3690572	total: 10.6s	remaining: 5.27s
668:	learn: 6.3667679	total: 10.6s	remaining: 5.25s
669:	learn: 6.3627755	total: 10.6s	remaining: 5.24s
670:	learn: 6.3553651	total: 10.6s	remaining: 5.22s
671:	learn: 6.3551924	total: 10.7s	remaining: 5.2s
672:	learn: 6.3548583	total: 10.7s	remaining: 5.19s
673:	learn: 6.3494231	total: 10.7s	remaining: 5.17s
674:	learn: 6.3313267	total: 10.7s	remaining: 5.16s
675:	learn: 6.3227093	total: 10.7s	remaining: 5.14s
676:	learn: 6.3205001	total: 10.7s	remaining: 5.13s
677:	learn: 6.3140421	total: 10.8s	remaining: 5.11s
678:	learn: 6.3014318	total: 10.8s	remaining: 5.09s
679:	learn: 6.3011323	total: 10.8s	remaining: 5.08s
680:	learn: 6.

...

460:	learn: 5.7443217	total: 7.78s	remaining: 9.09s
461:	learn: 5.7440575	total: 7.79s	remaining: 9.07s
462:	learn: 5.7400356	total: 7.81s	remaining: 9.06s
463:	learn: 5.7361311	total: 7.83s	remaining: 9.05s
464:	learn: 5.7331026	total: 7.85s	remaining: 9.03s
465:	learn: 5.7228311	total: 7.87s	remaining: 9.02s
466:	learn: 5.7211939	total: 7.89s	remaining: 9s
467:	learn: 5.7172943	total: 7.91s	remaining: 8.99s
468:	learn: 5.7035392	total: 7.92s	remaining: 8.97s
469:	learn: 5.6902474	total: 7.94s	remaining: 8.95s
470:	learn: 5.6896802	total: 7.96s	remaining: 8.94s
471:	learn: 5.6843416	total: 7.97s	remaining: 8.92s
472:	learn: 5.6775531	total: 7.99s	remaining: 8.91s
473:	learn: 5.6766719	total: 8.01s	remaining: 8.89s
474:	learn: 5.6678128	total: 8.03s	remaining: 8.87s
475:	learn: 5.6598489	total: 8.04s	remaining: 8.85s
476:	learn: 5.6546262	total: 8.06s	remaining: 8.84s
477:	learn: 5.6452869	total: 8.08s	remaining: 8.82s
478:	learn: 5.6327672	total: 8.1s	remaining: 8.8s
479:	learn: 5.616

630:	learn: 4.7179446	total: 10.7s	remaining: 6.25s
631:	learn: 4.7150282	total: 10.7s	remaining: 6.24s
632:	learn: 4.7083235	total: 10.7s	remaining: 6.22s
633:	learn: 4.7016120	total: 10.7s	remaining: 6.2s
634:	learn: 4.6908728	total: 10.8s	remaining: 6.18s
635:	learn: 4.6900010	total: 10.8s	remaining: 6.17s
636:	learn: 4.6730388	total: 10.8s	remaining: 6.15s
637:	learn: 4.6639347	total: 10.8s	remaining: 6.13s
638:	learn: 4.6565054	total: 10.8s	remaining: 6.11s
639:	learn: 4.6462539	total: 10.8s	remaining: 6.09s
640:	learn: 4.6449793	total: 10.9s	remaining: 6.08s
641:	learn: 4.6425022	total: 10.9s	remaining: 6.06s
642:	learn: 4.6351066	total: 10.9s	remaining: 6.04s
643:	learn: 4.6334040	total: 10.9s	remaining: 6.03s
644:	learn: 4.6245454	total: 10.9s	remaining: 6.01s
645:	learn: 4.6179643	total: 10.9s	remaining: 5.99s
646:	learn: 4.6074952	total: 11s	remaining: 5.98s
647:	learn: 4.6060977	total: 11s	remaining: 5.96s
648:	learn: 4.6002828	total: 11s	remaining: 5.94s
649:	learn: 4.59502

792:	learn: 3.9327233	total: 13.4s	remaining: 3.5s
793:	learn: 3.9292141	total: 13.4s	remaining: 3.49s
794:	learn: 3.9245251	total: 13.5s	remaining: 3.47s
795:	learn: 3.9189291	total: 13.5s	remaining: 3.45s
796:	learn: 3.9167790	total: 13.5s	remaining: 3.44s
797:	learn: 3.9162900	total: 13.5s	remaining: 3.42s
798:	learn: 3.9120407	total: 13.5s	remaining: 3.4s
799:	learn: 3.9069712	total: 13.5s	remaining: 3.39s
800:	learn: 3.9042575	total: 13.6s	remaining: 3.37s
801:	learn: 3.9028253	total: 13.6s	remaining: 3.35s
802:	learn: 3.8963970	total: 13.6s	remaining: 3.33s
803:	learn: 3.8928370	total: 13.6s	remaining: 3.32s
804:	learn: 3.8900300	total: 13.6s	remaining: 3.3s
805:	learn: 3.8895320	total: 13.6s	remaining: 3.28s
806:	learn: 3.8886194	total: 13.7s	remaining: 3.27s
807:	learn: 3.8799657	total: 13.7s	remaining: 3.25s
808:	learn: 3.8789738	total: 13.7s	remaining: 3.23s
809:	learn: 3.8763838	total: 13.7s	remaining: 3.22s
810:	learn: 3.8711149	total: 13.7s	remaining: 3.2s
811:	learn: 3.86

954:	learn: 3.3714718	total: 16.2s	remaining: 762ms
955:	learn: 3.3697427	total: 16.2s	remaining: 745ms
956:	learn: 3.3664416	total: 16.2s	remaining: 728ms
957:	learn: 3.3637356	total: 16.2s	remaining: 711ms
958:	learn: 3.3603404	total: 16.2s	remaining: 694ms
959:	learn: 3.3567089	total: 16.3s	remaining: 677ms
960:	learn: 3.3565189	total: 16.3s	remaining: 660ms
961:	learn: 3.3548890	total: 16.3s	remaining: 643ms
962:	learn: 3.3501539	total: 16.3s	remaining: 626ms
963:	learn: 3.3498167	total: 16.3s	remaining: 609ms
964:	learn: 3.3497055	total: 16.3s	remaining: 592ms
965:	learn: 3.3466313	total: 16.3s	remaining: 575ms
966:	learn: 3.3395978	total: 16.4s	remaining: 558ms
967:	learn: 3.3394687	total: 16.4s	remaining: 541ms
968:	learn: 3.3357728	total: 16.4s	remaining: 525ms
969:	learn: 3.3343971	total: 16.4s	remaining: 508ms
970:	learn: 3.3338334	total: 16.4s	remaining: 491ms
971:	learn: 3.3335075	total: 16.4s	remaining: 474ms
972:	learn: 3.3317741	total: 16.5s	remaining: 457ms
973:	learn: 

0

# Feature selection

In [30]:
def get_del_cols(df):
    # return the columns where no more than one value in the categorical features
    # exists in the test data - those features can be ignored
    cols_to_drop = []
    for col in df.select_dtypes(object).columns:
        col_vals = df[col].unique()
        n_vals = len(col_vals)
        n_irrelevant = 0
        for val in col_vals:
            if val not in df[col][m_train:].values:
                n_irrelevant += 1
        if n_irrelevant >= n_vals - 1:
            cols_to_drop.append(col)
    return cols_to_drop

In [31]:
cols_to_drop = get_del_cols(df)
df.drop(cols_to_drop, axis=1, inplace=True)
for col_to_drop in cols_to_drop:
    cols_cat.remove(col_to_drop)
    print("Dropped ", col_to_drop)

Dropped  Utilities


<!-- # Change ordinal columns to numeric, and encode accordingly -->

# Feature Engineering

## Features from the Internet - prefex FE0_

In [32]:
df["FE0_HighQualSF"] = df["GrLivArea"]+df["1stFlrSF"] + df["2ndFlrSF"]+0.5*df["GarageArea"]+0.5*df["TotalBsmtSF"]+1*df["MasVnrArea"]

df["FE0_SqFtPerRoom"] = df["GrLivArea"] / (df["TotRmsAbvGrd"] +
                                           df["FullBath"] +
                                           df["HalfBath"] +
                                           df["KitchenAbvGr"])

## My features - prefex FE1_

In [33]:
n_stories_dict = {
    "1Story": 1.0,
    "1.5Fin": 1.5,
    "1.5Unf": 1.5,
    "2Story": 2.0,
    "2.5Fin": 2.5,
    "2.5Unf": 2.5,
    "SFoyer": 2.0,
    "SLvl": 2.0,
}

df["FE1_n_stories"] = df["HouseStyle"].replace(n_stories_dict)
df["FE1_n_stories"].value_counts()

1.0    1471
2.0    1083
1.5     333
2.5      32
Name: FE1_n_stories, dtype: int64

In [34]:
df["FE1_age_sold"] = df["YrSold"] - df["YearBuilt"]
df["FE1_age_sold_Remod"] = df["YrSold"] - df["YearRemodAdd"]
df["FE1_GarageYrSold"] = df["YrSold"] - df["GarageYrBlt"]

In [35]:
df["CentralAir"] = df["CentralAir"].replace({"Y": 1, "N": 0})
df["CentralAir"].value_counts()

1    2723
0     196
Name: CentralAir, dtype: int64

In [36]:
df.select_dtypes(np.number).columns

Index(['LotArea', 'LotShape', 'LandSlope', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'ExterQual', 'ExterCond', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC',
       'CentralAir', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces',
       'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscVal', 'YrSold', 'LotShape_sq', 'LandSlope_sq',
       'ExterQualsq', 'ExterCondsq', 'BsmtQualsq', 'BsmtCondsq', 'HeatingQCsq',
       'KitchenQualsq', 'FireplaceQusq', 'GarageQualsq', 'GarageCondsq',
       'PoolQCsq', 'BsmtExposure_sq', 'BsmtFinType

In [37]:
df['FE1_total_area_hq'] = df["FE0_HighQualSF"] - df['LowQualFinSF']

df['FE1_Total_Home_Quality'] = df['OverallQual'] * df['OverallCond']
df['FE1_Total_bsmt_Quality'] = df['BsmtQual'] * df['BsmtCond']
df['FE1_Total_garage_Quality'] = df['GarageQual'] * df['GarageCond']

df["FE1_weighted_home_area"] = df["FE1_total_area_hq"] * df['FE1_Total_Home_Quality'] 
df["FE1_weighted_bsmt_area"] = df["TotalBsmtSF"] * df['FE1_Total_bsmt_Quality'] 
df["FE1_weighted_garage_area"] = df["GarageArea"] * df['FE1_Total_garage_Quality']
df["FE1_weighted_n_stories"] = df["FE1_n_stories"] * df['OverallQual']


# Split test and train

In [38]:
X_train, X_test = df[:m_train], df[m_train:]
y_train, _ = target[:m_train], target[m_train:]
n = X_train.shape[1]
X_train.shape, y_train.shape

((1460, 110), (1460,))

In [50]:
# df_ohe = pd.get_dummies(df)
# df_ohe[target_col] = target
# _ = setup(data=df_ohe[:m_train], target='SalePrice')
# best = compare_models()

# Training - CatBoost Grid Search 

In [51]:
# cat_features = df.select_dtypes(object).columns.to_list()
# model = CatBoostRegressor(random_state=10, cat_features=cat_features)
# kwargs = {
#     "n_estimators": [50 * n],
#     "max_depth": [8],
#     "subsample": [0.65],
#     "reg_lambda": [0.1], 
# }
# clf = GridSearchCV(model, kwargs, verbose=1, n_jobs=2)
# clf.fit(X_train, y_train)
# print(clf.best_score_)
# print(clf.best_params_)

# CatBoost Params:
    {'max_depth': 8, 'n_estimators': 5500, 'reg_lambda': 0.1, 'subsample': 0.65}

In [42]:
# clf.best_score_

0.8990345802296884

# Training - CatBoost with Best Parameters

In [40]:
model = CatBoostRegressor(max_depth=8, random_seed=10,
                          subsample=0.65, n_estimators=5500,
                          cat_features=cols_cat)
model.fit(X_train, y_train)

Learning rate set to 0.01087
0:	learn: 78838.8431751	total: 12.1ms	remaining: 1m 6s
1:	learn: 78232.8113257	total: 28ms	remaining: 1m 17s
2:	learn: 77689.1855244	total: 43.5ms	remaining: 1m 19s
3:	learn: 77166.0394430	total: 51.3ms	remaining: 1m 10s
4:	learn: 76633.3630922	total: 64.1ms	remaining: 1m 10s
5:	learn: 76108.2389016	total: 78.8ms	remaining: 1m 12s
6:	learn: 75578.3594320	total: 93.2ms	remaining: 1m 13s
7:	learn: 75041.8055713	total: 108ms	remaining: 1m 14s
8:	learn: 74520.1549516	total: 122ms	remaining: 1m 14s
9:	learn: 73978.8919168	total: 136ms	remaining: 1m 14s
10:	learn: 73444.8735833	total: 149ms	remaining: 1m 14s
11:	learn: 72924.8341253	total: 164ms	remaining: 1m 15s
12:	learn: 72376.8682996	total: 179ms	remaining: 1m 15s
13:	learn: 71881.1719499	total: 193ms	remaining: 1m 15s
14:	learn: 71399.3301119	total: 208ms	remaining: 1m 15s
15:	learn: 70951.6344114	total: 224ms	remaining: 1m 16s
16:	learn: 70500.3861291	total: 239ms	remaining: 1m 17s
17:	learn: 70017.4890337	

153:	learn: 33802.8808236	total: 2.31s	remaining: 1m 20s
154:	learn: 33675.1667577	total: 2.34s	remaining: 1m 20s
155:	learn: 33531.4090001	total: 2.35s	remaining: 1m 20s
156:	learn: 33415.0587398	total: 2.37s	remaining: 1m 20s
157:	learn: 33271.1866518	total: 2.39s	remaining: 1m 20s
158:	learn: 33151.0590638	total: 2.4s	remaining: 1m 20s
159:	learn: 33017.2713815	total: 2.42s	remaining: 1m 20s
160:	learn: 32889.1878358	total: 2.43s	remaining: 1m 20s
161:	learn: 32751.0420734	total: 2.45s	remaining: 1m 20s
162:	learn: 32616.3939662	total: 2.46s	remaining: 1m 20s
163:	learn: 32482.9166935	total: 2.48s	remaining: 1m 20s
164:	learn: 32365.4984399	total: 2.49s	remaining: 1m 20s
165:	learn: 32247.9113626	total: 2.51s	remaining: 1m 20s
166:	learn: 32129.2442369	total: 2.53s	remaining: 1m 20s
167:	learn: 31987.4565687	total: 2.54s	remaining: 1m 20s
168:	learn: 31870.8152222	total: 2.55s	remaining: 1m 20s
169:	learn: 31754.6941706	total: 2.57s	remaining: 1m 20s
170:	learn: 31641.3298528	total:

304:	learn: 22331.3457483	total: 4.61s	remaining: 1m 18s
305:	learn: 22289.2294821	total: 4.63s	remaining: 1m 18s
306:	learn: 22256.3898809	total: 4.65s	remaining: 1m 18s
307:	learn: 22201.5798612	total: 4.66s	remaining: 1m 18s
308:	learn: 22159.8690107	total: 4.68s	remaining: 1m 18s
309:	learn: 22113.9061676	total: 4.69s	remaining: 1m 18s
310:	learn: 22077.6491116	total: 4.71s	remaining: 1m 18s
311:	learn: 22039.4250421	total: 4.72s	remaining: 1m 18s
312:	learn: 21996.2407377	total: 4.74s	remaining: 1m 18s
313:	learn: 21960.4644214	total: 4.75s	remaining: 1m 18s
314:	learn: 21924.1084593	total: 4.77s	remaining: 1m 18s
315:	learn: 21888.5559732	total: 4.79s	remaining: 1m 18s
316:	learn: 21849.0397400	total: 4.8s	remaining: 1m 18s
317:	learn: 21814.9993644	total: 4.82s	remaining: 1m 18s
318:	learn: 21772.2773281	total: 4.83s	remaining: 1m 18s
319:	learn: 21736.2187567	total: 4.85s	remaining: 1m 18s
320:	learn: 21689.6745582	total: 4.87s	remaining: 1m 18s
321:	learn: 21647.6113652	total:

451:	learn: 18378.1643934	total: 6.91s	remaining: 1m 17s
452:	learn: 18359.5046489	total: 6.92s	remaining: 1m 17s
453:	learn: 18358.4875166	total: 6.93s	remaining: 1m 17s
454:	learn: 18341.9106365	total: 6.94s	remaining: 1m 16s
455:	learn: 18336.4727290	total: 6.96s	remaining: 1m 16s
456:	learn: 18313.7723761	total: 6.97s	remaining: 1m 16s
457:	learn: 18301.6343763	total: 6.99s	remaining: 1m 16s
458:	learn: 18285.2004129	total: 7.01s	remaining: 1m 16s
459:	learn: 18263.2307682	total: 7.02s	remaining: 1m 16s
460:	learn: 18239.5318692	total: 7.04s	remaining: 1m 16s
461:	learn: 18221.9663325	total: 7.05s	remaining: 1m 16s
462:	learn: 18205.5356597	total: 7.07s	remaining: 1m 16s
463:	learn: 18183.7871514	total: 7.08s	remaining: 1m 16s
464:	learn: 18169.7659685	total: 7.1s	remaining: 1m 16s
465:	learn: 18152.4435169	total: 7.12s	remaining: 1m 16s
466:	learn: 18136.4334399	total: 7.14s	remaining: 1m 16s
467:	learn: 18121.8153054	total: 7.16s	remaining: 1m 16s
468:	learn: 18105.6007401	total:

603:	learn: 16494.7958616	total: 9.21s	remaining: 1m 14s
604:	learn: 16476.0473232	total: 9.23s	remaining: 1m 14s
605:	learn: 16462.4031234	total: 9.24s	remaining: 1m 14s
606:	learn: 16448.8942828	total: 9.26s	remaining: 1m 14s
607:	learn: 16428.2391384	total: 9.28s	remaining: 1m 14s
608:	learn: 16415.5781398	total: 9.29s	remaining: 1m 14s
609:	learn: 16401.2998965	total: 9.31s	remaining: 1m 14s
610:	learn: 16394.8763275	total: 9.32s	remaining: 1m 14s
611:	learn: 16380.9408375	total: 9.34s	remaining: 1m 14s
612:	learn: 16364.7376697	total: 9.36s	remaining: 1m 14s
613:	learn: 16362.9457028	total: 9.36s	remaining: 1m 14s
614:	learn: 16352.8902311	total: 9.38s	remaining: 1m 14s
615:	learn: 16342.1579715	total: 9.39s	remaining: 1m 14s
616:	learn: 16339.4604924	total: 9.4s	remaining: 1m 14s
617:	learn: 16326.4409365	total: 9.41s	remaining: 1m 14s
618:	learn: 16316.7469437	total: 9.43s	remaining: 1m 14s
619:	learn: 16304.4476894	total: 9.45s	remaining: 1m 14s
620:	learn: 16300.4059843	total:

...

4756:	learn: 3727.4686715	total: 1m 21s	remaining: 12.7s
4757:	learn: 3727.3868727	total: 1m 21s	remaining: 12.7s
4758:	learn: 3727.0114218	total: 1m 21s	remaining: 12.7s
4759:	learn: 3725.4562359	total: 1m 21s	remaining: 12.7s
4760:	learn: 3724.7087143	total: 1m 21s	remaining: 12.7s
4761:	learn: 3724.6502149	total: 1m 21s	remaining: 12.6s
4762:	learn: 3723.2766629	total: 1m 21s	remaining: 12.6s
4763:	learn: 3723.1169622	total: 1m 21s	remaining: 12.6s
4764:	learn: 3721.2938186	total: 1m 21s	remaining: 12.6s
4765:	learn: 3720.7596062	total: 1m 21s	remaining: 12.6s
4766:	learn: 3720.3181699	total: 1m 21s	remaining: 12.6s
4767:	learn: 3718.6179249	total: 1m 21s	remaining: 12.5s
4768:	learn: 3718.5316066	total: 1m 21s	remaining: 12.5s
4769:	learn: 3717.2715071	total: 1m 21s	remaining: 12.5s
4770:	learn: 3715.0878765	total: 1m 21s	remaining: 12.5s
4771:	learn: 3714.3260344	total: 1m 21s	remaining: 12.5s
4772:	learn: 3713.9446810	total: 1m 21s	remaining: 12.5s
4773:	learn: 3711.6109550	total

4902:	learn: 3576.1728839	total: 1m 23s	remaining: 10.2s
4903:	learn: 3574.5964601	total: 1m 23s	remaining: 10.2s
4904:	learn: 3574.0448798	total: 1m 24s	remaining: 10.2s
4905:	learn: 3572.7246423	total: 1m 24s	remaining: 10.2s
4906:	learn: 3571.2251678	total: 1m 24s	remaining: 10.2s
4907:	learn: 3570.7070168	total: 1m 24s	remaining: 10.1s
4908:	learn: 3570.4083035	total: 1m 24s	remaining: 10.1s
4909:	learn: 3569.6018993	total: 1m 24s	remaining: 10.1s
4910:	learn: 3567.8290979	total: 1m 24s	remaining: 10.1s
4911:	learn: 3566.3634491	total: 1m 24s	remaining: 10.1s
4912:	learn: 3565.7002022	total: 1m 24s	remaining: 10.1s
4913:	learn: 3565.6413491	total: 1m 24s	remaining: 10s
4914:	learn: 3565.5200422	total: 1m 24s	remaining: 10s
4915:	learn: 3565.4618866	total: 1m 24s	remaining: 10s
4916:	learn: 3564.7641878	total: 1m 24s	remaining: 9.98s
4917:	learn: 3564.6844707	total: 1m 24s	remaining: 9.97s
4918:	learn: 3564.2161475	total: 1m 24s	remaining: 9.95s
4919:	learn: 3563.8607927	total: 1m 2

5057:	learn: 3433.9464275	total: 1m 26s	remaining: 7.58s
5058:	learn: 3433.8986305	total: 1m 26s	remaining: 7.56s
5059:	learn: 3432.0685065	total: 1m 26s	remaining: 7.54s
5060:	learn: 3431.4197557	total: 1m 26s	remaining: 7.53s
5061:	learn: 3429.9715847	total: 1m 26s	remaining: 7.51s
5062:	learn: 3427.0748103	total: 1m 26s	remaining: 7.49s
5063:	learn: 3426.4893061	total: 1m 26s	remaining: 7.47s
5064:	learn: 3426.4359884	total: 1m 26s	remaining: 7.46s
5065:	learn: 3426.3870294	total: 1m 26s	remaining: 7.44s
5066:	learn: 3426.0211273	total: 1m 26s	remaining: 7.42s
5067:	learn: 3424.7312694	total: 1m 26s	remaining: 7.4s
5068:	learn: 3423.9119967	total: 1m 26s	remaining: 7.39s
5069:	learn: 3422.5196668	total: 1m 26s	remaining: 7.37s
5070:	learn: 3421.1850527	total: 1m 26s	remaining: 7.35s
5071:	learn: 3420.8141456	total: 1m 26s	remaining: 7.34s
5072:	learn: 3419.7645613	total: 1m 26s	remaining: 7.32s
5073:	learn: 3419.0160997	total: 1m 26s	remaining: 7.3s
5074:	learn: 3418.1063459	total: 

5213:	learn: 3294.6329562	total: 1m 29s	remaining: 4.9s
5214:	learn: 3294.0138431	total: 1m 29s	remaining: 4.89s
5215:	learn: 3293.2990173	total: 1m 29s	remaining: 4.87s
5216:	learn: 3292.8998514	total: 1m 29s	remaining: 4.85s
5217:	learn: 3291.3362749	total: 1m 29s	remaining: 4.83s
5218:	learn: 3289.2988091	total: 1m 29s	remaining: 4.82s
5219:	learn: 3288.8486451	total: 1m 29s	remaining: 4.8s
5220:	learn: 3287.8714020	total: 1m 29s	remaining: 4.78s
5221:	learn: 3286.4428021	total: 1m 29s	remaining: 4.77s
5222:	learn: 3286.3325822	total: 1m 29s	remaining: 4.75s
5223:	learn: 3285.5261365	total: 1m 29s	remaining: 4.73s
5224:	learn: 3284.7716527	total: 1m 29s	remaining: 4.71s
5225:	learn: 3284.1003009	total: 1m 29s	remaining: 4.7s
5226:	learn: 3283.2395136	total: 1m 29s	remaining: 4.68s
5227:	learn: 3282.2222474	total: 1m 29s	remaining: 4.66s
5228:	learn: 3281.8892531	total: 1m 29s	remaining: 4.65s
5229:	learn: 3281.4988750	total: 1m 29s	remaining: 4.63s
5230:	learn: 3280.1575691	total: 1

5359:	learn: 3171.8762883	total: 1m 31s	remaining: 2.4s
5360:	learn: 3171.1665024	total: 1m 31s	remaining: 2.38s
5361:	learn: 3171.1351309	total: 1m 31s	remaining: 2.37s
5362:	learn: 3171.0950058	total: 1m 31s	remaining: 2.35s
5363:	learn: 3170.7045898	total: 1m 31s	remaining: 2.33s
5364:	learn: 3169.9942283	total: 1m 32s	remaining: 2.31s
5365:	learn: 3168.6140519	total: 1m 32s	remaining: 2.3s
5366:	learn: 3166.7246267	total: 1m 32s	remaining: 2.28s
5367:	learn: 3165.6965692	total: 1m 32s	remaining: 2.26s
5368:	learn: 3165.6646838	total: 1m 32s	remaining: 2.25s
5369:	learn: 3165.5843293	total: 1m 32s	remaining: 2.23s
5370:	learn: 3165.4905211	total: 1m 32s	remaining: 2.21s
5371:	learn: 3165.0385166	total: 1m 32s	remaining: 2.19s
5372:	learn: 3164.4546491	total: 1m 32s	remaining: 2.18s
5373:	learn: 3163.9514421	total: 1m 32s	remaining: 2.16s
5374:	learn: 3162.9767057	total: 1m 32s	remaining: 2.14s
5375:	learn: 3162.2748021	total: 1m 32s	remaining: 2.13s
5376:	learn: 3161.6162632	total: 

In [42]:
print(model.best_score_)
fe = model.get_feature_importance(prettified=True)
fe.head(20)

{'learn': {'RMSE': 3058.639198018039}}


,Feature Id,Importances
0,FE1_weighted_home_area,14.147725
1,OverallQual,7.225429
2,FE1_total_area_hq,6.771710
3,FE0_HighQualSF,5.898317
4,Neighborhood,4.273528
5,FE1_weighted_bsmt_area,3.679384
6,BsmtFinSF1,3.340338
7,GrLivArea,2.554121
8,LotArea,2.553034
9,TotalBsmtSF,1.869597


In [43]:
sub_name = "data/submission_15.csv"
pd.DataFrame(model.predict(X_test), 
            index=range(1461, len(df)+1), 
            columns=['SalePrice']).reset_index().\
            rename(columns={'index': 'id'}).to_csv(sub_name, index=False)

# Score: 0.12114